In [1]:
from libs.configuration import configure
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

from shared.spark_config import create_spark_config

env = configure()
conf = create_spark_config("M2_Processors.aircraft.tier1")

{'DATASTORAGE_AZURE_ACCESSKEY': 'dYj3PCGRyYHYR9pRo2xPa9BQ7t/sUonACJ29wcPdpa+1IC70rMbZmpfpHJJWJtk7OeASL+eQDIWq+AStGg3CCA==',
 'DATASTORAGE_AZURE_ACCOUNTNAME': 'icebergpe6t5cug7x9f45p8',
 'KAFKA_BOOTSTRAP_SERVERS': 'localhost:9092',
 'KAFKA_SCHEMA_REGISTRY_URL': 'http://localhost:8081',
 'KAFKA_SESSION_TIMEOUT_MS': '45000',
 'KAFKA_TOPIC_RAW_AIRCRAFT': 'raw.aircraft',
 'KAFKA_TOPIC_RAW_AIRLINE': 'raw.airline',
 'KAFKA_TOPIC_RAW_AIRPORT': 'raw.airport',
 'KAFKA_TOPIC_RAW_FLIGHT_SIGNAL': 'raw.flight_signal',
 'KAFKA_TOPIC_T1_AIRCRAFT': 't1.aircraft',
 'KAFKA_TOPIC_T1_AIRLINE': 't1.airline',
 'KAFKA_TOPIC_T1_AIRPORT': 't1.airport',
 'KAFKA_TOPIC_T1_FLIGHT_SIGNAL': 't1.flight_signal',
 'KAFKA_TOPIC_T2_AIRCRAFT': 't2.aircraft',
 'KAFKA_TOPIC_T2_AIRLINE': 't2.airline',
 'KAFKA_TOPIC_T2_AIRPORT': 't2.airport',
 'KAFKA_TOPIC_T2_FLIGHT_SIGNAL': 't2.flight_signal',
 'POSTGRE_JDBC_URL': 'jdbc:postgresql://localhost:5432/iceberg-catalog',
 'POSTGRE_PASSWORD': 'grebeci',
 'POSTGRE_USER': 'iceberg',
 

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [115]:
df = spark.read.format("iceberg").load("dev.raw.aircrafts")
df.schema

StructType([StructField('created_ts', TimestampType(), True), StructField('icao24', StringType(), True), StructField('registration', StringType(), True), StructField('manufacturericao', StringType(), True), StructField('model', StringType(), True), StructField('typecode', StringType(), True), StructField('serialnumber', StringType(), True), StructField('operator', StringType(), True), StructField('operatorcallsign', StringType(), True), StructField('operatoricao', StringType(), True), StructField('operatoriata', StringType(), True), StructField('owner', StringType(), True), StructField('registered', DateType(), True), StructField('reguntil', DateType(), True), StructField('built', DateType(), True), StructField('engines', StringType(), True), StructField('modes', BooleanType(), True), StructField('adsb', BooleanType(), True), StructField('acars', BooleanType(), True)])

In [ ]:
df.describe().show()

In [116]:
df = (
    df.withColumn(
        "registration", F.regexp_replace("registration", r"(\t)|[^a-zA-Z0-9\-\.\ \+\/\?]+", "")
    )
    .withColumn("model", F.regexp_replace("model", r"\t+", ""))
    .withColumn("serialnumber", F.regexp_replace("serialnumber", r"\s+", ""))
    .withColumn("operator", F.regexp_replace("operator", r"\t", ""))
    .withColumn("operatorcallsign", F.regexp_replace("operatorcallsign", r"\t", ""))
    .withColumn("operatoricao", F.regexp_replace("operatoricao", r"\t", ""))
    .withColumn("operatoriata", F.regexp_replace("operatoriata", r"\t", ""))
    .withColumn("owner", F.regexp_replace("owner", r"\t", ""))
    .withColumn("engines", F.regexp_replace("owner", r"\t", ""))
)

In [117]:
df = df.replace(
    to_replace={"n/a": None, "N/A": None},
    subset=[
        "registration",
        "serialnumber",
        "operator",
        "operatorcallsign",
        "operatoricao",
        "operatoriata",
        "owner",
        "engines",
    ],
)

In [118]:
df.select("icao24").describe().show()

+-------+--------+
|summary|  icao24|
+-------+--------+
|  count|  519999|
|   mean|Infinity|
| stddev|     NaN|
|    min|  000000|
|    max|  fffffe|
+-------+--------+



In [119]:
df.select("manufacturericao").describe().show()

+-------+----------------+
|summary|manufacturericao|
+-------+----------------+
|  count|          428796|
|   mean|            NULL|
| stddev|            NULL|
|    min|          3XTRIM|
|    max|             ZLT|
+-------+----------------+



In [136]:
df = df.withColumn(
    "registration",
    F.when(F.col("registration").contains("?"), None).otherwise(F.col("registration")),
).withColumn("registration", F.upper("registration"))

In [121]:
df = (
    df.withColumn("model", F.trim("model"))
    .withColumn("model", F.regexp_replace("model", r"^\"", ""))
    .withColumn("model", F.regexp_replace("model", r"\"$", ""))
    .withColumn("model", F.regexp_replace("model", r"\"{2}", '"'))
)

In [122]:
df = (
    df.withColumn("operator", F.trim("operator"))
    .withColumn("operator", F.regexp_replace("operator", r"^\"", ""))
    .withColumn("operator", F.regexp_replace("operator", r"\"$", ""))
    .withColumn("operator", F.regexp_replace("operator", r"\"{2}", '"'))
)

In [123]:
df = df.withColumn("operatorcallsign", F.trim("operatorcallsign")).withColumn(
    "operatorcallsign",
    F.when(F.col("operatorcallsign") == "", None)
    .when(F.col("operatorcallsign") == "-", None)
    .otherwise(F.col("operatorcallsign")),
)

In [124]:
df = df.withColumn("operatoricao", F.trim("operatoricao")).withColumn(
    "operatoricao", F.when(F.col("operatoricao") == "-", None).otherwise(F.col("operatoricao"))
)

In [126]:
df = df.withColumn("operatoriata", F.trim("operatoriata")).withColumn(
    "operatoriata", F.when(F.col("operatoriata") == "-", None).otherwise(F.col("operatoriata"))
)

In [137]:
df.describe().show()

+-------+--------+-----------------+----------------+--------------------+--------+------------+--------------------+----------------+-----------------+------------------+----------------+----------------+
|summary|  icao24|     registration|manufacturericao|               model|typecode|serialnumber|            operator|operatorcallsign|     operatoricao|      operatoriata|           owner|         engines|
+-------+--------+-----------------+----------------+--------------------+--------+------------+--------------------+----------------+-----------------+------------------+----------------+----------------+
|  count|  519999|           516525|          428796|              440424|  479944|      437117|               23687|           40066|            41392|              8056|          435911|          435911|
|   mean|Infinity|          79841.0|            NULL|            Infinity|   737.0|    Infinity|                NULL|            30.0|         200204.0| 4.647058823529412|     

In [141]:
df = df.where(F.col("icao24").isNotNull())

In [142]:
df = df.withColumn("created_ts", F.current_timestamp()).withColumn(
    "updated_ts", F.current_timestamp()
)

In [143]:
df.show()

+--------------------+------+------------+----------------+--------------------+--------+------------+--------+----------------+------------+------------+--------------------+----------+----------+----------+--------------------+-----+-----+-----+--------------------+
|          created_ts|icao24|registration|manufacturericao|               model|typecode|serialnumber|operator|operatorcallsign|operatoricao|operatoriata|               owner|registered|  reguntil|     built|             engines|modes| adsb|acars|          updated_ts|
+--------------------+------+------------+----------------+--------------------+--------+------------+--------+----------------+------------+------------+--------------------+----------+----------+----------+--------------------+-----+-----+-----+--------------------+
|2024-01-27 23:16:...|aa3487|       N757F|        RAYTHEON|                 A36|    BE36|      E-3121|    NULL|            NULL|        NULL|        NULL|Vintage Aircraft Llc|      NULL|2027-01

In [144]:
spark.sql(
    """
CREATE TABLE IF NOT EXISTS dev.tier1.aircrafts (
    created_ts timestamp,
    updated_ts timestamp,
    icao24 string,
    registration string,
    manufacturer_icao string,
    model string,
    doc8643_icao string,
    serial_no string,
    operator string,
    operator_callsign string,
    operator_icao string,
    operator_iata string,
    owner string,
    registered_date date,
    registered_until_date date,
    built_date date,
    engines string,
    modes boolean,
    adsb boolean,
    acars boolean
)
USING iceberg
"""
).show()

++
||
++
++



In [145]:
df = df.withColumnsRenamed(
    {
        # "icao24": "icao24",
        # "registration": "registration",
        "manufacturericao": "manufacturer_icao",
        # "model": "model",
        "typecode": "doc8643_icao",
        "serialnumber": "serial_no",
        # "operator": "operator",
        "operatorcallsign": "operator_callsign",
        "operatoricao": "operator_icao",
        "operatoriata": "operator_iata",
        # "owner": "owner",
        "registered": "registered_date",
        "reguntil": "registered_until_date",
        "built": "built_date",
        # "engines": "engines",
        # "modes": "modes",
        # "adsb": "adsb",
        # "acars": "acars",
    }
)

In [146]:
df.writeTo("dev.tier1.aircrafts").append()